# Word Embeddings in MySQL

This example uses the official MySQL Connector within Python3 to store and retrieve various amounts of Word Embeddings.

We will use a local MySQL database running as a Docker Container for testing purposes. To start the database run:

```
docker run -ti --rm --name ohmysql -e MYSQL_ROOT_PASSWORD=mikolov -e MYSQL_DATABASE=embeddings -p 3306:3306 mysql:5.7
```

In [ ]:
import mysql.connector
import io
import time
import numpy
import plotly

# Dummy Embeddings

For testing purposes we will use randomly generated numpy arrays as dummy embbeddings. Since loading gigabytes of gensim KeyedVectors is slow and annoying.

In [ ]:
def embeddings(n=1000, dim=300):
    """
    Yield n tuples of random numpy arrays of *dim* length indexed by *n*
    """
    idx = 0
    while idx < n:
        yield (str(idx), numpy.random.rand(dim))
        idx += 1

# Conversion Functions

In [ ]:
def adapt_array(array):
    """
    Using the numpy.save function to save a binary version of the array,
    and BytesIO to catch the stream of data and convert it into a BLOB.
    """
    out = io.BytesIO()
    numpy.save(out, array)
    out.seek(0)

    return out.read()

def convert_array(blob):
    """
    Using BytesIO to convert the binary version of the array back into a numpy array.
    """
    out = io.BytesIO(blob)
    out.seek(0)

    return numpy.load(out)

In [ ]:
connection = mysql.connector.connect(user='root', password='mikolov',
                              host='127.0.0.1',
                              database='embeddings')

cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS `embeddings` (`key` TEXT, `embedding` BLOB);')

In [ ]:
%%time
for key, emb in embeddings():
    arr = adapt_array(emb)
    cursor.execute('INSERT INTO `embeddings` (`key`, `embedding`) VALUES (%s, %s);', (key, arr))

In [ ]:
%%time
for key, emb in embeddings():
    cursor.execute('SELECT embedding FROM `embeddings` WHERE `key`=%s;', (key,))
    data = cursor.fetchone()
    arr = convert_array(data[0])

# Sample some data

In [ ]:
write_times = []
read_times = []
# counts = [500, 1000, 5000, 10000, 50000, 100000]
counts = numpy.logspace(2,5,dtype='int')

for c in counts:
    cursor.execute('DROP TABLE IF EXISTS `embeddings`;')
    cursor.execute('CREATE TABLE IF NOT EXISTS `embeddings` (`key` TEXT, `embedding` BLOB);')

    start_time_write = time.time()
    for key, emb in embeddings():
        arr = adapt_array(emb)
        cursor.execute('INSERT INTO `embeddings` (`key`, `embedding`) VALUES (%s, %s);', (key, arr))
    write_times.append(time.time() - start_time_write)
    
    start_time_read = time.time()
    for key, emb in embeddings():
        cursor.execute('SELECT embedding FROM `embeddings` WHERE `key`=%s;', (key,))
        data = cursor.fetchone()
        arr = convert_array(data[0])
    read_times.append(time.time() - start_time_read)
    
print('DONE')

# Results

In [ ]:
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Scatter(
    x = write_times,
    y = counts,
    mode = 'markers'
)
layout = plotly.graph_objs.Layout(title="MySQL Write Times",
                xaxis=dict(title='Time in Seconds'),
                yaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic-scatter')

In [ ]:
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Scatter(
    x = read_times,
    y = counts,
    mode = 'markers'
)
layout = plotly.graph_objs.Layout(title="MySQL Read Times",
                xaxis=dict(title='Time in Seconds'),
                yaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic-scatter')